In [1]:
# This notebook does some checks that ST_Distance is computing the distance correctly in meters, using checks from Google maps

# Some reference values:  Distance from London WC2N 5AQ 51.5092488,-0.0978045 to York:  53.9598567,-1.0874146  YO32 4TW  280.42km
# Distance from Liverpool  L2 5UZ 53.4061444,-2.9873456 to Norwich NR3 1ES 52.6261985,1.2968884 299.48km

In [2]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/postgresql/base.py:2505: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [42]:
# select * from
# (select * from a left join b on)

sql = """

select london, york, st_distance(london,york)/1000 as distance_km from
(select geom as london from tt.all_postcodes where postcode = 'WC2N5AQ') as a,
(select geom as york from tt.all_postcodes where postcode = 'YO324TW') as b


"""
import pandas as pd 
pd.read_sql(sql, conn)

,london,york,distance_km
0,0101000020346C0000000000008A2F2041000000002003...,0101000020346C000000000000781C1C4100000000C0C2...,283.242904


In [74]:
sql = """
select norwich, liverpool, st_distance(norwich,liverpool)/1000 as distance_km from
(select geom as norwich from tt.all_postcodes where postcode = 'NR3 1ES') as a,
(select geom as liverpool from tt.all_postcodes where postcode = 'L2  5UZ') as b
"""
pd.read_sql(sql, conn)

,norwich,liverpool,distance_km
0,0101000020346C000000000000220423410000000090DB...,0101000020346C0000000000009C68144100000000B0D6...,300.056858


In [75]:
# Check distance is also working in the stations data   According to Google maps this is 137.44km
sql = """
select farringdon, coventry, st_distance(farringdon,coventry)/1000 as distance_km from
(select geom as farringdon from tt.all_stations where station_name='Farringdon') as a,
(select geom as coventry from tt.all_stations where station_name = 'Coventry') as b
"""
pd.read_sql(sql, conn)

,farringdon,coventry,distance_km
0,0101000020346C00000000000018372041000000002033...,0101000020346C000000000000306F1A410000000070FC...,137.665432


In [80]:
# Finally check it's working for the staff locations data
# Distance should be 160ish

# 
sql = """
select person1, person1pc, person2, person2pc, st_distance(person1,person2)/1000 as distance_km from
(select geom as person1, postcode as person1pc from tt.staff_locations order by random() limit 1) as a,
(select geom as person2, postcode as person2pc from tt.staff_locations order by random() limit 1) as b
"""
# pd.read_sql(sql, conn)

,person1,person1pc,person2,person2pc,distance_km
0,0101000020346C000000000000ECFF1D4100000000187B...,NG310EN,0101000020346C000000000000606B2041000000002825...,E14 0RZ,161.010907


In [86]:
# Finally, check units of meaurement of ST_Buffer

# Distance from Farringdon station to Elephant and Castle is 2.93 km
sql = """
select nlc, station_name, st_distance((select geom from tt.all_stations where station_name = 'Farringdon'), a.geom)/1000 as distance
from tt.all_stations as a
where 
st_contains(
    ST_Buffer((select geom from tt.all_stations where station_name = 'Farringdon'),5000),
    a.geom)

"""
pd.read_sql(sql, conn)

,nlc,station_name,?column?
0,577,Farringdon,0.000000
1,1022,Haggerston,3.138849
2,1023,Hoxton,2.603118
3,1039,Rotherhithe,4.482210
4,1082,Shadwell,3.728697
5,1085,Wapping,4.051632
6,1555,St.Pancras,1.722021
7,1024,Shoreditch High Street,2.226127
8,1429,Dalston (Kingsland),3.893995
9,1440,Camden Road,3.359025
